<table style="width:100%; background-color: #EBF5FB">
  <tr>
    <td style="border: 1px solid #CFCFCF">
      <b>Time series: Processing Notebook</b>
      <ul>
        <li><a href="main.ipynb">Main Notebook</a></li>
        <li>Processing Notebook</li>
      </ul>
      <br>This Notebook is part of the <a href="http://data.open-power-system-data.org/time_series">Time series Data Package</a> of <a href="http://open-power-system-data.org">Open Power System Data</a>.
    </td>
  </tr>
</table>

# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Introductory-Notes" data-toc-modified-id="Introductory-Notes-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introductory Notes</a></div><div class="lev1 toc-item"><a href="#Settings" data-toc-modified-id="Settings-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Settings</a></div><div class="lev2 toc-item"><a href="#Set-version-number-and-recent-changes" data-toc-modified-id="Set-version-number-and-recent-changes-21"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Set version number and recent changes</a></div><div class="lev2 toc-item"><a href="#Import-Python-libraries" data-toc-modified-id="Import-Python-libraries-22"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Import Python libraries</a></div><div class="lev2 toc-item"><a href="#Set-directories" data-toc-modified-id="Set-directories-23"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Set directories</a></div><div class="lev2 toc-item"><a href="#Set-up-a-log" data-toc-modified-id="Set-up-a-log-24"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Set up a log</a></div><div class="lev2 toc-item"><a href="#Select-timerange" data-toc-modified-id="Select-timerange-25"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Select timerange</a></div><div class="lev2 toc-item"><a href="#Select-download-source" data-toc-modified-id="Select-download-source-26"><span class="toc-item-num">2.6&nbsp;&nbsp;</span>Select download source</a></div><div class="lev2 toc-item"><a href="#Select-subset" data-toc-modified-id="Select-subset-27"><span class="toc-item-num">2.7&nbsp;&nbsp;</span>Select subset</a></div><div class="lev1 toc-item"><a href="#Download" data-toc-modified-id="Download-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Download</a></div><div class="lev2 toc-item"><a href="#Automatic-download-(for-all-sources-except-Energinet.dk)" data-toc-modified-id="Automatic-download-(for-all-sources-except-Energinet.dk)-31"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Automatic download (for all sources except Energinet.dk)</a></div><div class="lev2 toc-item"><a href="#Manual-download-(Energinet.dk)" data-toc-modified-id="Manual-download-(Energinet.dk)-32"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Manual download (Energinet.dk)</a></div><div class="lev1 toc-item"><a href="#Read" data-toc-modified-id="Read-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Read</a></div><div class="lev2 toc-item"><a href="#Preparations" data-toc-modified-id="Preparations-41"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Preparations</a></div><div class="lev2 toc-item"><a href="#Reading-loop" data-toc-modified-id="Reading-loop-42"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Reading loop</a></div><div class="lev2 toc-item"><a href="#Save-raw-data" data-toc-modified-id="Save-raw-data-43"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Save raw data</a></div><div class="lev1 toc-item"><a href="#Processing" data-toc-modified-id="Processing-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Processing</a></div><div class="lev2 toc-item"><a href="#Missing-data-handling" data-toc-modified-id="Missing-data-handling-51"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Missing data handling</a></div><div class="lev3 toc-item"><a href="#Interpolation" data-toc-modified-id="Interpolation-511"><span class="toc-item-num">5.1.1&nbsp;&nbsp;</span>Interpolation</a></div><div class="lev2 toc-item"><a href="#Country-specific-calculations" data-toc-modified-id="Country-specific-calculations-52"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Country specific calculations</a></div><div class="lev3 toc-item"><a href="#Calculate-onshore-wind-generation-for-German-TSOs" data-toc-modified-id="Calculate-onshore-wind-generation-for-German-TSOs-521"><span class="toc-item-num">5.2.1&nbsp;&nbsp;</span>Calculate onshore wind generation for German TSOs</a></div><div class="lev3 toc-item"><a href="#Calculate-aggregate-wind-capacity-for-germany-(on-+-offshore)" data-toc-modified-id="Calculate-aggregate-wind-capacity-for-germany-(on-+-offshore)-522"><span class="toc-item-num">5.2.2&nbsp;&nbsp;</span>Calculate aggregate wind capacity for germany (on + offshore)</a></div><div class="lev3 toc-item"><a href="#Aggregate-German-data-from-individual-TSOs-and-calculate-availabilities/profiles" data-toc-modified-id="Aggregate-German-data-from-individual-TSOs-and-calculate-availabilities/profiles-523"><span class="toc-item-num">5.2.3&nbsp;&nbsp;</span>Aggregate German data from individual TSOs and calculate availabilities/profiles</a></div><div class="lev2 toc-item"><a href="#Create-hourly-data-from-15'-data" data-toc-modified-id="Create-hourly-data-from-15'-data-53"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Create hourly data from 15' data</a></div><div class="lev2 toc-item"><a href="#Fill-columns-not-retrieved-directly-from-TSO-webites-with--ENTSO-E-Transparency-data" data-toc-modified-id="Fill-columns-not-retrieved-directly-from-TSO-webites-with--ENTSO-E-Transparency-data-54"><span class="toc-item-num">5.4&nbsp;&nbsp;</span>Fill columns not retrieved directly from TSO webites with  ENTSO-E Transparency data</a></div><div class="lev2 toc-item"><a href="#Insert-a-column-with-Central-European-(Summer-)time" data-toc-modified-id="Insert-a-column-with-Central-European-(Summer-)time-55"><span class="toc-item-num">5.5&nbsp;&nbsp;</span>Insert a column with Central European (Summer-)time</a></div><div class="lev1 toc-item"><a href="#Create-metadata" data-toc-modified-id="Create-metadata-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Create metadata</a></div><div class="lev1 toc-item"><a href="#Write-data-to-disk" data-toc-modified-id="Write-data-to-disk-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Write data to disk</a></div><div class="lev2 toc-item"><a href="#Limit-time-range" data-toc-modified-id="Limit-time-range-71"><span class="toc-item-num">7.1&nbsp;&nbsp;</span>Limit time range</a></div><div class="lev2 toc-item"><a href="#Different-shapes" data-toc-modified-id="Different-shapes-72"><span class="toc-item-num">7.2&nbsp;&nbsp;</span>Different shapes</a></div><div class="lev2 toc-item"><a href="#Write-to-SQL-database" data-toc-modified-id="Write-to-SQL-database-73"><span class="toc-item-num">7.3&nbsp;&nbsp;</span>Write to SQL-database</a></div><div class="lev2 toc-item"><a href="#Write-to-Excel" data-toc-modified-id="Write-to-Excel-74"><span class="toc-item-num">7.4&nbsp;&nbsp;</span>Write to Excel</a></div><div class="lev2 toc-item"><a href="#Write-to-CSV" data-toc-modified-id="Write-to-CSV-75"><span class="toc-item-num">7.5&nbsp;&nbsp;</span>Write to CSV</a></div><div class="lev2 toc-item"><a href="#Write-checksums.txt" data-toc-modified-id="Write-checksums.txt-76"><span class="toc-item-num">7.6&nbsp;&nbsp;</span>Write checksums.txt</a></div>

# Introductory Notes

This Notebook handles missing data, performs calculations and aggragations and creates the output files.

# Settings

## Set version number and recent changes
Executing this script till the end will create a new version of the data package.
The Version number specifies the local directory for the data <br>
We include a note on what has been changed.

In [ ]:
version = '2017-07-09'
changes = '''include ENTSO E transparency, RTE, APG'''

## Import Python libraries

This section: load libraries and set up a log.

Note that the download module makes use of the [pycountry](https://pypi.python.org/pypi/pycountry) library that is not part of Anaconda. Install it with with `pip install pycountry` from the command line.

In [ ]:
# Python modules
from datetime import datetime, date, timedelta, time
import pandas as pd
import numpy as np
import logging
import json
import sqlite3
import yaml
import itertools
import os
import pytz
import hashlib
from shutil import copyfile

# Skripts from time-series repository
from timeseries_scripts.read import read
from timeseries_scripts.download import download
from timeseries_scripts.imputation import find_nan
from timeseries_scripts.imputation import resample_markers
from timeseries_scripts.make_json import make_json

# Reload modules with execution of any code, to avoid having to restart
# the kernel after editing timeseries_scripts
%load_ext autoreload
%autoreload 2

## Set directories

In [ ]:
# make sure the working directory is this file's directory
try:
    os.chdir(home_path)
except NameError:
    home_path = os.path.realpath('.')

sources_yaml_path = os.path.join(home_path, 'input', 'sources.yml')
areas_csv_path = os.path.join(home_path, 'input', 'areas.csv')
data_path = os.path.join('E:', os.sep, 'time series', version, 'original_data')
out_path = os.path.join('E:', os.sep, 'time series', version)
temp_path = os.path.join('E:', os.sep, 'time series', 'temp')
os.makedirs(out_path, exist_ok=True)
os.makedirs(temp_path, exist_ok=True)
# change to temp directory
os.chdir(temp_path)
os.getcwd()

## Set up a log

In [ ]:
FORMAT = '%(asctime)s %(levelname)s %(message)s'
DATEFORMAT = '%Y-%m-%d %H:%M:%S'
formatter = logging.Formatter(fmt=FORMAT, datefmt=DATEFORMAT)
logging.basicConfig(level=logging.INFO,
                    format=FORMAT,
                    datefmt=DATEFORMAT)

logfile = logging.FileHandler('log1.log')
logfile.setFormatter(formatter)
logfile.setLevel(logging.INFO)
logger = logging.getLogger(__name__)
logger.addHandler(logfile)

logger.setLevel(logging.INFO)
# For more detailed logging messages, replace 'INFO' with 'DEBUG'
# (May slow down computation).

## Select timerange

This section: select the time range and the data sources for download and read. Default: all data sources implemented, full time range available.

**Source parameters** are specified in [input/sources.yml](input/sources.yml), which describes, for each source, the datasets (such as wind and solar generation) alongside all the parameters necessary to execute the downloads.

The option to perform downloading and reading of subsets is for testing only. To be able to run the script succesfully until the end, all sources have to be included, or otherwise the script will run into errors (i.e. the step where aggregate German timeseries are caculated requires data from all four German TSOs to be loaded).

In order to do this, specify the beginning and end of the interval for which to attempt the download.

Type `None` to download all available data.

In [ ]:
start_from_user = date(2006, 1, 1)
end_from_user = date(2017, 5, 31)

## Select download source

Instead of downloading from the sources, the complete raw data can be downloaded as a zip file from the OPSD Server. Advantages are:
- much faster download
- back up of raw data in case it is deleted from the server at the original source

In order to do this, specify an archive version to use the raw data from that version that has been cached on the OPSD server as input. All data from that version will be downloaded - timerange and subset will be ignored.

Type `None` to download directly from the original sources.

In [ ]:
archive_version = None  # i.e. '2016-07-14'

## Select subset

Optionally, specify a subset to download/read.<br>
The next cell prints the available sources and datasets.<br>

In [ ]:
with open(sources_yaml_path, 'r') as f:
    sources = yaml.load(f.read())
for k, v in sources.items():
    print(yaml.dump({k: list(v.keys())}, default_flow_style=False))

Copy from its output and paste to following cell to get the right format.<br>
Type `subset = None` to include all data.

In [ ]:
subset = yaml.load('''
insert_source_here:
- insert_dataset1_from_that_source_here
- insert_dataset2_here
more_sources:
- more_data_sets
''')

# need to exclude Elia data due to unclear copyright situation
exclude = yaml.load('''
- Elia
''')  
subset = {}

In [ ]:
subset = yaml.load('''
PSE:
- wind
''')


Now eliminate sources and variables not in subset.

In [ ]:
with open(sources_yaml_path, 'r') as f:
    sources = yaml.load(f.read())
if subset:  # eliminate sources and variables not in subset
    sources = {source_name: {k: v
                             for k, v in sources[source_name].items()
                             if k in variable_list}
               for source_name, variable_list in subset.items()}
if exclude:  # eliminate sources and variables in exclude
    sources = {source_name: variable_dict
               for source_name, variable_dict in sources.items()
               if not source_name in exclude}

# Download

This section: download data. Takes about 1 hour to run for the complete data set (`subset=None`).

First, a data directory is created on your local computer. Then, download parameters for each data source are defined, including the URL. These parameters are then turned into a YAML-string. Finally, the download is executed file by file.

Each file is saved under it's original filename. Note that the original file names are often not self-explanatory (called "data" or "January"). The files content is revealed by its place in the directory structure.

## Automatic download (for all sources except Energinet.dk)

In [ ]:
download(sources, data_path,
         archive_version=archive_version,
         start_from_user=start_from_user,
         end_from_user=end_from_user)

## Manual download (Energinet.dk)

[Energinet.dk](https://en.energinet.dk/Electricity/Energy-data) data needs to be downloaded manually [here](http://osp.energinet.dk/_layouts/Markedsdata/framework/integrations/markedsdatatemplate.aspx). The website seems as of 2017-07 to be only available in Danish.


Check The Boxes as specified below, then press the "Get extract"-button at the end of the page.
You will receive a file `Market Data.xls` of about 50 MB. Open the file in Excel. There will be a warning from Excel saying that file extension and content are in conflict. Select "open anyways" and and save the file as `.xlsx`. This will compress the file by ~80%, resulting in faster processing afterwards. 

In order to be found by the read-function, place the downloaded file in the following subdirectory:<br>
` \original_data\Energinet.dk\prices_wind_solar\2005-01-01_2017-12-31\ ` <br>

**Boxes to check:**

- Period
    - Get data from
        - 01-01-2005
    - To
        - Today
    - all months
- Data columns
    - Elspot Price, Currency Code/MWh
        - DK-West
        - DK-East
        - Norway
        - Sweden (SE)
        - Sweden (SE3)
        - Sweden (SE4)
        - DE European Power Exchange
    - Production and consumption, MWh/h
        - DK-West: Wind power production
        - DK-West: Solar cell production (estimated)
        - DK-East: Wind power production
        - DK-East: Solar cell production (estimated)
        - DK: Wind power production (onshore)
        - DK: Wind power production (offshore)
- Data format
    - Currency code
        - EUR
    - Decimal format
        - English number Format (period as decimal separator)
    - Date format
        - Other date format(YYYY-MM-DD)
    - Recieve to
        - Excel




# Read

This section: Read each downloaded file into a pandas-DataFrame and merge data from different sources if it has the same time resolution. Takes ~15 minutes to run.

## Preparations

Set the title of the rows at the top of the data used to store metadata internally. The order of this list determines the order of the levels in the resulting output.

In [ ]:
headers = ['region', 'variable', 'attribute', 'source', 'web', 'unit']

Read a prepared table containing meta data on the geographical areas

In [ ]:
areas = pd.read_csv(areas_csv_path)

View the areas table

In [ ]:
areas.loc[areas['OPSD area'].notnull(),:'EIC'].fillna('')

## Reading loop

Loop through sources and variables to do the reading.
First read the originla CSV, Excel etc. files into pandas DataFrames.

In [ ]:
areas = pd.read_csv(areas_csv_path)

# For each source in the source dictionary
for source_name, source_dict in sources.items():
    # For each variable from source_name
    for variable_name, param_dict in source_dict.items():
#        variable_dir = os.path.join(data_path, source_name, variable_name)
        res_list = param_dict['resolution']
        url = param_dict['web']
        for res_key in res_list:
            df = read(data_path, areas, source_name, variable_name,
                      url, res_key, headers,
                      start_from_user=start_from_user,
                      end_from_user=end_from_user)

            os.makedirs(res_key, exist_ok=True)
            filename = '_'.join([source_name, variable_name]) + '.pickle'
            df.to_pickle(os.path.join(res_key, filename))

Then combine the DataFrames that have the same temporal resolution

In [ ]:
# Create a dictionary of empty DataFrames to be populated with data
data_sets = {'15min': pd.DataFrame(), '30min': pd.DataFrame(), '60min': pd.DataFrame()}
entso_e = {'15min': pd.DataFrame(), '30min': pd.DataFrame(), '60min': pd.DataFrame()}
for res_key in data_sets.keys():
    for filename in os.listdir(res_key):
        source_name = filename.split('_')[0]
        if subset and not source_name in subset.keys():
            continue
        logger.info(filename)
        df_portion = pd.read_pickle(os.path.join(res_key, filename))

        if source_name == 'ENTSO-E Transparency FTP':
            dfs = entso_e
        else:
            dfs = data_sets

        if dfs[res_key].empty:
            dfs[res_key] = df_portion
        elif not df_portion.empty:
            dfs[res_key] = dfs[res_key].combine_first(df_portion)
        else:
            logger.warning(filename + 'was empty')

In [ ]:
for res_key, df in data_sets.items():
    logger.info(res_key + ': %s', df.shape)

In [ ]:
for res_key, df in entso_e.items():
    logger.info(res_key + ': %s', df.shape)

Display some rows of the dataframes to get a first impression of the data.

In [ ]:
data_sets['60min'].head()

## Save raw data

Save the DataFrames created by the read function to disk. This way you have the raw data to fall back to if something goes wrong in the ramainder of this notebook without having to repeat the previos steps.

In [ ]:
data_sets['15min'].to_pickle('raw_data_15.pickle')
data_sets['30min'].to_pickle('raw_data_30.pickle')
data_sets['60min'].to_pickle('raw_data_60.pickle')
entso_e['15min'].to_pickle('raw_entso_e_15.pickle')
entso_e['30min'].to_pickle('raw_entso_e_30.pickle')
entso_e['60min'].to_pickle('raw_entso_e_60.pickle')

Load the DataFrames saved above

In [ ]:
data_sets = {}
data_sets['15min'] = pd.read_pickle('raw_data_15.pickle')
data_sets['30min'] = pd.read_pickle('raw_data_30.pickle')
data_sets['60min'] = pd.read_pickle('raw_data_60.pickle')
entso_e['15min'] = pd.read_pickle('raw_entso_e_15.pickle')
entso_e['30min'] = pd.read_pickle('raw_entso_e_30.pickle')
entso_e['60min'] = pd.read_pickle('raw_entso_e_60.pickle')

# Processing

This section: missing data handling, aggregation of sub-national to national data, aggregate 15'-data to 60'-resolution. Takes 30 minutes to run.

## Missing data handling

### Interpolation

Patch missing data. At this stage, only small gaps (up to 2 hours) are filled by linear interpolation. This catched most of the missing data due to daylight savings time transitions, while leaving bigger gaps untouched

The exact locations of missing data are stored in the `nan_table` DataFrames.

Where data has been interpolated, it is marked in a new column `comment`. For eaxample the comment `solar_DE-transnetbw_generation;` means that in the original data, there is a gap in the solar generation timeseries from TransnetBW in the time period where the marker appears.

Patch the datasets and display the location of missing Data in the original data. Takes ~5 minutes to run.

In [ ]:
nan_tables = {}
for res_key, df in data_sets.items():
    data_sets[res_key], nan_tables[res_key] = find_nan(
        df, res_key, headers, patch=True)

In [ ]:
for res_key, df in entso_e.items():
    entso_e[res_key], nan_tables[res_key + ' ENTSO-E'] = find_nan(
        df, res_key, headers, patch=True)

Execute this to see an example of where the data has been patched.

In [ ]:
data_sets['15min'][data_sets['15min']['interpolated_values'].notnull()].tail()

Display the table of regions of missing values

In [ ]:
nan_tables['60min']

You can export the NaN-tables to Excel in order to inspect where there are NaNs

In [ ]:
writer = pd.ExcelWriter('NaN_table.xlsx')
for res_key, df in nan_tables.items():
    df.to_excel(writer, res_key)
writer.save()

Save/Load the patched data sets

In [ ]:
data_sets['15min'].to_pickle('patched_15.pickle')
data_sets['30min'].to_pickle('patched_30.pickle')
data_sets['60min'].to_pickle('patched_60.pickle')
entso_e['15min'].to_pickle('patched_entso_e_15.pickle')
entso_e['30min'].to_pickle('patched_entso_e_30.pickle')
entso_e['60min'].to_pickle('patched_entso_e_60.pickle')

In [ ]:
#data_sets = {}
data_sets['15min'] = pd.read_pickle('patched_15.pickle')
data_sets['30min'] = pd.read_pickle('patched_30.pickle')
data_sets['60min'] = pd.read_pickle('patched_60.pickle')
entso_e['15min'] = pd.read_pickle('patched_entso_e_15.pickle')
entso_e['30min'] = pd.read_pickle('patched_entso_e_30.pickle')
entso_e['60min'] = pd.read_pickle('patched_entso_e_60.pickle')

## Country specific calculations

### Calculate onshore wind generation for German TSOs

For 50 Hertz, it is already in the data.
For TenneT, it is calculated by substracting offshore from total generation.
For Amprion and TransnetBW, onshore wind generation is just total wind generation.
Takes <1 second to run.

In [ ]:
# Some of the following operations require the Dataframes to be lexsorted in
# the columns
for res_key, df in data_sets.items():
    df.sort_index(axis=1, inplace=True)

In [ ]:
for area, source in zip(['DE_amprion', 'DE_tennet', 'DE_transnetbw'],
                        ['Amprion', 'TenneT', 'TransnetBW']):
    new_col_header = {
        'variable': 'wind_onshore',
        'region': '{tso}',
        'attribute': 'generation',
        'source': 'own calculation',
        'web': '',
        'unit': 'MW'
    }

    if area == 'DE_tennet':
        colname = ('DE_tennet', 'wind_offshore', 'generation', 'TenneT')
        offshore = data_sets['15min'].loc[:, colname]
    else:
        offshore = 0

    data_sets['15min'][
        tuple(new_col_header[level].format(tso=area) for level in headers)
    ] = (data_sets['15min'][(area, 'wind', 'generation', source)] - offshore)

    # Sort again
    data_sets['15min'].sort_index(axis=1, inplace=True)

### Calculate aggregate wind capacity for germany (on + offshore)

Apart from being interesting on it's own, this is also required to calculate an aggregated wind-profile for Germany

In [ ]:
new_col_header = {
    'variable': 'wind',
    'region': 'DE',
    'attribute': 'capacity',
    'source': 'BNetzA and Netztransparenz.de',
    'web': 'http://data.open-power-system-data.org/renewable_power_plants',
    'unit': 'MW'
}
new_col_header = tuple(new_col_header[level] for level in headers)

data_sets['15min'][new_col_header] = (
    data_sets['15min']
    .loc[:, ('DE', ['wind_onshore', 'wind_offshore'], 'capacity')]
    .sum(axis=1, skipna=False))

# Sort again
data_sets['15min'].sort_index(axis=1, inplace=True)

### Aggregate German data from individual TSOs and calculate availabilities/profiles

The wind and solar in-feed data for the 4 German balancing areas is summed up and stored in in new columns, which are then used to calculate profiles, that is, the share of wind/solar capacity producing at a given time. The column headers are created in the fashion introduced in the read script. Takes 5 seconds to run.

In [ ]:
for tech in ['solar', 'wind', 'wind_onshore', 'wind_offshore']:
    for attribute in ['generation']:  # we could also include 'forecast'
        sum_col = data_sets['15min'].loc(axis=1)[(
            ['DE_50hertz', 'DE_amprion', 'DE_tennet', 'DE_transnetbw'],
            tech, attribute)].sum(axis=1, skipna=False).to_frame()

        # Create a new MultiIndex and append aggregated data to the dataset
        new_col_header = {
            'variable': '{tech}',
            'region': 'DE',
            'attribute': '{attribute}',
            'source': 'own calculation',
            'web': '',
            'unit': 'MW'
        }
        tuples = [tuple(new_col_header[level].format(
            tech=tech, attribute=attribute) for level in headers)]
        sum_col.columns = pd.MultiIndex.from_tuples(tuples, names=headers)
        data_sets['15min'] = data_sets['15min'].combine_first(sum_col)

        if not attribute == 'generation':
            continue
        
        # Calculate the profile column
        profile_col = (sum_col.values * 100 /
                       data_sets['15min']['DE', tech, 'capacity'])
        
        # Create a new MultiIndex and append aggregated data to the dataset
        new_col_header = {
            'variable': '{tech}',
            'region': 'DE',
            'attribute': 'profile',
            'source': 'own calculation',
            'web': '',
            'unit': 'percent'
        }
        tuples = [tuple(new_col_header[level].format(tech=tech)
                        for level in headers)]
        profile_col.columns = pd.MultiIndex.from_tuples(tuples, names=headers)
        data_sets['15min'] = data_sets['15min'].combine_first(profile_col)

## Create hourly data from 15' data

Some data comes in 15-minute intervals (i.e. German renewable generation), other in 60-minutes (i.e. load data from ENTSO-E and Prices). We resample the 15-minute data to hourly resolution and append it to the 60-minutes dataset.

The marker column is resampled separately in such a way that all information on where data has been interpolated is preserved.

The `.resample('H').mean()` methods calculates the means from the values for 4 quarter hours [:00, :15, :30, :45] of an hour values, inserts that for :00 and drops the other 3 entries. Takes 15 seconds to run.

In [ ]:
# Condense marker column from 15 to 60 minutes resolutution
marker_15 = data_sets['15min']['interpolated_values'].groupby(
    pd.Grouper(freq='60Min', closed='left', label='left')
    ).agg(resample_markers)
marker_15 = marker_15.reindex(data_sets['60min'].index)

# Glue condensed 15 min marker onto 60 min marker
marker_60 = data_sets['60min']['interpolated_values']
both_markers = marker_15.notnull() & marker_60.notnull()
only_15 = marker_15.notnull() & marker_60.isnull()
data_sets['60min'].loc[both_markers, 'interpolated_values'] = (
    marker_60.str.cat(others=marker_15, sep=' | '))
data_sets['60min'].loc[only_15, 'interpolated_values'] = marker_15

resampled = data_sets['15min'].resample('H').mean()
data_sets['60min'] = data_sets['60min'].combine_first(resampled)

## Fill columns not retrieved directly from TSO webites with  ENTSO-E Transparency data

In [ ]:
for res_key, df in entso_e.items():
    resampled = (df.drop('DE_AT_LU', axis=1, errors='ignore')
                 .resample('H').mean())
    entso_e['60min'] = entso_e['60min'].combine_first(resampled)
    
    # combine with TSO data
    if data_sets[res_key].empty:
        data_sets[res_key] = df
    else:
        data_cols = data_sets[res_key].columns.droplevel(['source', 'web', 'unit'])
        add_cols = [col for col in df.columns if not col[:3] in data_cols]
        add_cols = pd.MultiIndex.from_tuples(add_cols, names=headers)
        data_sets[res_key] = data_sets[res_key].combine_first(df[add_cols])

## Insert a column with Central European (Summer-)time

The index column of th data sets defines the start of the timeperiod represented by each row of that data set in **UTC** time. We include an additional column for the **CE(S)T** Central European (Summer-) Time, as this might help aligning the output data with other data sources.

In [ ]:
info_cols = {'utc': 'utc_timestamp',
             'cet': 'cet_cest_timestamp',
             'marker': 'interpolated_values'}

In [ ]:
for res_key, df in data_sets.items():
    if df.empty:
        continue
    df.index.rename(info_cols['utc'], inplace=True)
    df.insert(0, info_cols['cet'],
              df.index.tz_localize('UTC').tz_convert('Europe/Brussels'))

Create a final savepoint

In [ ]:
data_sets['15min'].to_pickle('final_15.pickle')
data_sets['30min'].to_pickle('final_30.pickle')
data_sets['60min'].to_pickle('final_60.pickle')

In [ ]:
data_sets = {}
data_sets['15min'] = pd.read_pickle('final_15.pickle')
data_sets['30min'] = pd.read_pickle('final_30.pickle')
data_sets['60min'] = pd.read_pickle('final_60.pickle')

# Create metadata

This section: create the metadata, both general and column-specific. All metadata we be stored as a JSON file. Takes <1s to run.

In [ ]:
# change to out_path directory
os.chdir(out_path)
os.getcwd()

In [ ]:
make_json(data_sets, info_cols, version, changes, headers, areas)

# Write data to disk

This section: Save as [Data Package](http://data.okfn.org/doc/tabular-data-package) (data in CSV, metadata in JSON file). All files are saved in the directory of this notebook. Alternative file formats (SQL, XLSX) are also exported. Takes about 1 hour to run.

## Limit time range
Cut off the data outside of [start_from_user:end_from_user]

In [ ]:
for res_key, df in data_sets.items():
    # First, convert userinput to UTC time to conform with data_set.index
    if start_from_user:
        start_from_user = (
            pytz.timezone('Europe/Brussels')
            .localize(datetime.combine(start_from_user, time()))
            .astimezone(pytz.timezone('UTC')))
    if end_from_user:
        end_from_user = (
            pytz.timezone('Europe/Brussels')
            .localize(datetime.combine(end_from_user, time()))
            .astimezone(pytz.timezone('UTC'))
            # appropriate offset to inlude the end of period
            + timedelta(days=1, minutes=-int(res_key[:2])))
    # Then cut off the data_set
    data_sets[res_key] = df.loc[start_from_user:end_from_user, :]

## Different shapes

Data are provided in three different "shapes": 
- SingleIndex (easy to read for humans, compatible with datapackage standard, small file size)
  - Fileformat: CSV, SQLite
- MultiIndex (easy to read into GAMS, not compatible with datapackage standard, small file size)
  - Fileformat: CSV, Excel
- Stacked (compatible with data package standard, large file size, many rows, too many for Excel) 
  - Fileformat: CSV

The different shapes need to be created internally befor they can be saved to files. Takes about 1 minute to run.

In [ ]:
data_sets_singleindex = {}
data_sets_multiindex = {}
data_sets_stacked = {}
for res_key, df in data_sets.items():
    if df.empty:
        continue

    for col_name, col in df.iteritems():
        if not (col_name[0] in info_cols.values() or
                col_name[2] == 'profile'):
            df[col_name] = col.round(0)

    # MultIndex
    data_sets_multiindex[res_key + '_multiindex'] = df
    
    # SingleIndex
    df_singleindex = df.copy()
    # use first 3 levels of multiindex to create singleindex
    df_singleindex.columns = [
        col[0] if col[0] in info_cols.values()
        else '_'.join([level for level in col[0:3] if not level == ''])
        for col in df.columns.values]

    data_sets_singleindex[res_key + '_singleindex'] = df_singleindex

    # Stacked
    stacked = df.copy()
    stacked.drop(info_cols['cet'], axis=1, inplace=True)
    stacked.columns = stacked.columns.droplevel(['source', 'web', 'unit'])
    stacked = stacked.transpose().stack(dropna=True).to_frame(name='data')
    data_sets_stacked[res_key + '_stacked'] = stacked

## Write to SQL-database

This file format is required for the filtering function on the OPSD website. This takes about 30 seconds to complete.

In [ ]:
for res_key, df in data_sets_singleindex.items():
    table = 'time_series_' + res_key
    df = df.copy()
    df.index = df.index.strftime('%Y-%m-%dT%H:%M:%SZ')
    df[info_cols['cet']] = (
        df[info_cols['cet']].dt.strftime('%Y-%m-%dT%H:%M:%S%z'))
    df.to_sql(table, sqlite3.connect('time_series.sqlite'),
              if_exists='replace', index_label=info_cols['utc'])

## Write to Excel

Writing the full tables to Excel takes extremely long. As a workaround, only the first 5 rows are exported. The rest of the data can than be inserted manually from the `_multindex.csv` files.

In [ ]:
writer = pd.ExcelWriter('time_series1.xlsx')
for res_key, df in data_sets_multiindex.items():
    df.head().to_excel(writer, res_key.split('_')[0], float_format='%.2f',
                       merge_cells=True)
    # merge_cells=False doesn't work properly with multiindex
writer.save()

## Write to CSV

This takes about 10 minutes to complete.

In [ ]:
# itertoools.chain() allows iterating over multiple dicts at once
for res_stacking_key, df in itertools.chain(
#        data_sets_singleindex.items(),
#        data_sets_multiindex.items(),
        data_sets_stacked.items()
    ):
    # convert the format of the cet_cest-timestamp to ISO-8601
    if not (res_stacking_key.split('_')[1] == 'stacked'
            or type(df.iloc[0,0]) == str):
        df.iloc[:,0] = df.iloc[:,0].dt.strftime('%Y-%m-%dT%H:%M:%S%z')
    filename = 'time_series_' + res_stacking_key + '.csv'
    df.to_csv(filename, float_format='%.2f',
              date_format='%Y-%m-%dT%H:%M:%SZ')

## Write checksums.txt

We publish SHA-checksums for the outputfiles on GitHub to allow verifying the integrity of outputfiles on the OPSD server.

In [ ]:
def get_sha_hash(path, blocksize=65536):
    sha_hasher = hashlib.sha256()
    with open(path, 'rb') as f:
        buffer = f.read(blocksize)
        while len(buffer) > 0:
            sha_hasher.update(buffer)
            buffer = f.read(blocksize)
        return sha_hasher.hexdigest()

files = os.listdir(out_path)

# Create checksums.txt in the output directory
with open('checksums.txt', 'w') as f:
    for file_name in files:
        if file_name.split('.')[-1] in ['csv', 'sqlite', 'xlsx']:
            file_hash = get_sha_hash(file_name)
            f.write('{},{}\n'.format(file_name, file_hash))

# Copy the file to root directory from where it will be pushed to GitHub,
# leaving a copy in the version directory for reference
copyfile('checksums.txt', os.path.join(home_path, 'checksums.txt'))